In [1]:
%run header.py
%matplotlib inline

from utils import sample_from_interactions
from utils import divide_train_test
from utils import from_interactions_to_coo
from utils import combine_with_identity

In [2]:
exp_path = '/nmnt/x04-hdd/boris_temp/SGIMC_IMC/movielens/'

In [3]:
PROBLEM = "classification" if False else "regression"

step_fn = step_qaadmm

if PROBLEM == "classification":
    QAObjectiveLoss = QAObjectiveLogLoss
else:
    QAObjectiveLoss = QAObjectiveL2Loss  # QAObjectiveHuberLoss

## Data loading

We find out, that problem cannot be solved with only side-information features.

X and Y combined with the identity matrices by default.

In [4]:
interactions = np.load(exp_path + 'I.npy')
X = np.load(exp_path + 'X.npy')
Y = np.load(exp_path + 'Y.npy')
X = combine_with_identity(X)
Y = combine_with_identity(Y)

n = len(np.unique(interactions[0])) # users 
m = len(np.unique(interactions[1])) # items

R_full = from_interactions_to_coo(interactions, shape=(n,m))
full_mask = R_full.toarray() > 0

## SGIMC test on Movielens

In [5]:
C_ridge = 1e-4
C_group = 3e-4
C_lasso = 10 * C_group
C = (C_lasso, C_group, C_ridge)

eta = 5e0

step_kwargs = {
    "C": C,                 # the regularizr constants (C_lasso, C_group, C_ridge)
    "eta": eta,             # the eta of the ADMM (larger - faster but more unstable)
    "rtol": 1e-4,           # the relative tolerance for stopping the ADMM
    "atol": 1e-7,           # the absolute tolerance
    "method": "cg",         # the method to use in Sub_0
    "n_iterations": 2,      # the number of iterations of the inner ADMM
}

n_iter = 100
Ks = np.arange(2,16,1)

### Random split

In [6]:
elements = [0.4, 0.5, 0.6, 0.7, 0.8]

In [7]:
sgimc_losses = []
for elem in tqdm(elements):
    
    Ks_losses = []
    for K in Ks:
        
        random_losses = []
        for i in [7,42]:
            
            interaction_train, interaction_test = sample_from_interactions(interactions, elem, seed=i)
            R_train = from_interactions_to_coo(interaction_train, shape=(n,m)).tocsr()
            train_mask = R_train.toarray() > 0
            test_mask = from_interactions_to_coo(interaction_test, shape=(n,m)).toarray() > 0

            problem = IMCProblem(QAObjectiveLoss, X, Y, R_train, n_threads=8)

            W_0 = random_state.normal(size=(X.shape[1], K))
            H_0 = random_state.normal(size=(Y.shape[1], K))

            W, H = W_0.copy(), H_0.copy()

            W, H = imc_descent(problem, W, H,
                               step_fn,                  # the inner optimization
                               step_kwargs=step_kwargs,  # asrtguments for the inner optimizer
                               n_iterations=n_iter,      # the number of outer iterations (Gauss-Siedel)
                               n_init_iterations=0,
                               return_history=True,      # Record the evolution of the matrices (W, H)
                               rtol=1e-5,                # relative stopping tolerance for the outer iterations
                               atol=1e-7,                # absolute tolerance
                               verbose=False,            # show the progress bar
                               check_product=True,       # use the product W H' for stopping
                               )
            
            R_hat = get_prediction(X, W, H, Y)
            random_losses.append(relative_loss(R_full.toarray(), R_hat, mask=test_mask))
            
        Ks_losses.append(np.mean(random_losses))
    sgimc_losses.append(np.min(Ks_losses))

100%|██████████| 5/5 [17:17<00:00, 210.79s/it]


In [10]:
sgimc_losses = np.array(sgimc_losses)
np.save(exp_path + 'sgimc_random.npy', sgimc_losses)

### Cold-start sampling

In [11]:
elements = [0.7, 0.75, 0.8, 0.85, 0.9]

In [12]:
sgimc_losses = []
for elem in tqdm(elements):
    
    Ks_losses = []
    for K in Ks:
        
        random_losses = []
        for i in [7,42]:
            
            I_oo, I_on, I_no, I_nn = divide_train_test(interactions, shape=(n,m), train_size=elem, seed=i)
            R_oo = from_interactions_to_coo(I_oo, shape=(n,m)).tocsr()
            mask_no = from_interactions_to_coo(I_no, shape=(n,m)).toarray() > 0

            problem = IMCProblem(QAObjectiveLoss, X, Y, R_oo, n_threads=8)

            W_0 = random_state.normal(size=(X.shape[1], K))
            H_0 = random_state.normal(size=(Y.shape[1], K))

            W, H = W_0.copy(), H_0.copy()

            W, H = imc_descent(problem, W, H,
                               step_fn,                  # the inner optimization
                               step_kwargs=step_kwargs,  # asrtguments for the inner optimizer
                               n_iterations=n_iter,      # the number of outer iterations (Gauss-Siedel)
                               n_init_iterations=0,
                               return_history=True,      # Record the evolution of the matrices (W, H)
                               rtol=1e-5,                # relative stopping tolerance for the outer iterations
                               atol=1e-7,                # absolute tolerance
                               verbose=False,            # show the progress bar
                               check_product=True,       # use the product W H' for stopping
                               )
            
            R_hat = get_prediction(X, W, H, Y)
            random_losses.append(relative_loss(R_full.toarray(), R_hat, mask=mask_no))
            
        Ks_losses.append(np.mean(random_losses))
    sgimc_losses.append(np.min(Ks_losses))

  0%|          | 0/5 [00:00<?, ?it/s]/nmnt/media/home/boris/env3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
100%|██████████| 5/5 [52:35<00:00, 632.98s/it]


In [14]:
sgimc_losses = np.array(sgimc_losses)
np.save(exp_path + 'sgimc_coldstart.npy', sgimc_losses)

In [16]:
elem_random = [0.4, 0.5, 0.6, 0.7, 0.8]
elem_coldstart = [0.7, 0.75, 0.8, 0.85, 0.9]

In [17]:
sgimc_random = np.load(exp_path + 'sgimc_random.npy')
imc_random = np.load(exp_path + 'imc_random.npy')
mf_random = np.load(exp_path + 'mf_random.npy')

print(elem_random)
print(sgimc_random)
print(imc_random)
print(mf_random)

[0.4, 0.5, 0.6, 0.7, 0.8]
[0.17741381 0.16399969 0.1302093  0.09347925 0.08855051]
[0.08179756 0.08246713 0.07103076 0.06488327 0.06230304]
[0.49912426 0.40738828 0.25007069 0.06655666 0.06541152]


In [8]:
sgimc_losses

[0.6102101415713154,
 0.4639162882313005,
 0.29762718190317433,
 0.26213268239100085,
 0.11268782604736596]

In [18]:
sgimc_coldstart = np.load(exp_path + 'sgimc_coldstart.npy')
imc_coldstart = np.load(exp_path + 'imc_coldstart.npy')
mf_coldstart = np.load(exp_path + 'mf_coldstart.npy')

print(elem_coldstart)
print(sgimc_coldstart)
print(imc_coldstart)
print(mf_coldstart)

[0.7, 0.75, 0.8, 0.85, 0.9]
[0.16530622 0.14961579 0.20616977 0.14622519 0.18784847]
[0.09620888 0.09391305 0.09666226 0.09177917 0.08917045]
[0.99920574 0.99668088 0.99550499 0.99544602 0.99067397]
